<h2 align="center">AtliQ Hotels Data Analysis Project<h2>

In [1]:
import pandas as pd

***
### ==> 1. Data Import and Data Exploration
***

### Datasets
We have 5 csv file 

   - dim_date.csv  
   - dim_hotels.csv
   - dim_rooms.csv
   - fact_aggregated_bookings
   - fact_bookings.csv

**Read bookings data in a datagrame**

In [2]:
df_bookings = pd.read_csv('datasets/fact_bookings.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'datasets/fact_bookings.csv'

**Explore bookings data**

In [ ]:
df_bookings.head()

In [ ]:
df_bookings.shape

In [ ]:
df_bookings.room_category.unique()

In [ ]:
df_bookings.booking_platform.unique()

In [ ]:
df_bookings.booking_platform.value_counts()

In [ ]:
df_bookings.booking_platform.value_counts().plot(kind="bar")

In [ ]:
df_bookings.describe()

**Read rest of the files**

In [ ]:
df_date = pd.read_csv('datasets/dim_date.csv')
df_hotels = pd.read_csv('datasets/dim_hotels.csv')
df_rooms = pd.read_csv('datasets/dim_rooms.csv')
df_agg_bookings = pd.read_csv('datasets/fact_aggregated_bookings.csv')

In [ ]:
df_hotels.shape

In [ ]:
df_hotels.head(3)

In [ ]:
df_hotels.category.value_counts()

In [ ]:
df_hotels.city.value_counts().plot(kind="bar")

In [ ]:
df_agg_bookings.head(3)

***
### ==> 2. Data Cleaning
***

In [ ]:
df_bookings.describe()

**(1) Clean invalid guests**

In [ ]:
df_bookings[df_bookings.no_guests<=0]

As you can see above, number of guests having less than zero value represents data error. We can ignore these records.

In [ ]:
df_bookings = df_bookings[df_bookings.no_guests>0]

In [ ]:
df_bookings.shape

**(2) Outlier removal in revenue generated**

In [ ]:
df_bookings.revenue_generated.min(), df_bookings.revenue_generated.max()

In [ ]:
df_bookings.revenue_generated.mean(), df_bookings.revenue_generated.median()

In [ ]:
avg, std = df_bookings.revenue_generated.mean(), df_bookings.revenue_generated.std()

In [ ]:
higher_limit = avg + 3*std
higher_limit

In [ ]:
lower_limit = avg - 3*std
lower_limit

In [ ]:
df_bookings[df_bookings.revenue_generated<=0]

In [ ]:
df_bookings[df_bookings.revenue_generated>higher_limit]

In [ ]:
df_bookings = df_bookings[df_bookings.revenue_generated<=higher_limit]
df_bookings.shape

In [ ]:
df_bookings.revenue_realized.describe()

In [ ]:
higher_limit = df_bookings.revenue_realized.mean() + 3*df_bookings.revenue_realized.std()
higher_limit

In [ ]:
df_bookings[df_bookings.revenue_realized>higher_limit]

One observation we can have in above dataframe is that all rooms are RT4 which means presidential suit. Now since RT4 is a luxurious room it is likely their rent will be higher. To make a fair analysis, we need to do data analysis only on RT4 room types

In [ ]:
df_bookings[df_bookings.room_category=="RT4"].revenue_realized.describe()

In [ ]:
# mean + 3*standard deviation
23439+3*9048

Here higher limit comes to be 50583 and in our dataframe above we can see that max value for revenue realized is 45220. Hence we can conclude that there is no outlier and we don't need to do any data cleaning on this particular column

In [ ]:
df_bookings[df_bookings.booking_id=="May012216558RT213"]

In [ ]:
df_bookings.isnull().sum()

Total values in our dataframe is 134576. Out of that 77899 rows has null rating. Since there are many rows with null rating, we should not filter these values. Also we should not replace this rating with a median or mean rating etc 

***
### ==> 3. Data Transformation
***

**Create occupancy percentage column**

In [ ]:
df_agg_bookings.head(3)

In [ ]:
df_agg_bookings['occ_pct'] = df_agg_bookings.apply(lambda row: row['successful_bookings']/row['capacity'], axis=1)

You can use following approach to get rid of SettingWithCopyWarning

In [ ]:
new_col = df_agg_bookings.apply(lambda row: row['successful_bookings']/row['capacity'], axis=1)
df_agg_bookings = df_agg_bookings.assign(occ_pct=new_col.values)
df_agg_bookings.head(3)

Convert it to a percentage value

In [ ]:
df_agg_bookings['occ_pct'] = df_agg_bookings['occ_pct'].apply(lambda x: round(x*100, 2))
df_agg_bookings.head(3)

In [ ]:
df_bookings.head()

In [ ]:
df_agg_bookings.info()

There are various types of data transformations that you may have to perform based on the need. Few examples of data transformations are,

1. Creating new columns
1. Normalization
1. Merging data
1. Aggregation

***
### ==> 4. Insights Generation
***

**1. What is an average occupancy rate in each of the room categories?**

In [ ]:
df_agg_bookings.head(3)

In [ ]:
df_agg_bookings.groupby("room_category")["occ_pct"].mean()

I don't understand RT1, RT2 etc. Print room categories such as Standard, Premium, Elite etc along with average occupancy percentage

In [ ]:
df = pd.merge(df_agg_bookings, df_rooms, left_on="room_category", right_on="room_id")
df.head(4)

In [ ]:
df.drop("room_id",axis=1, inplace=True)
df.head(4)

In [ ]:
df.groupby("room_class")["occ_pct"].mean()

In [ ]:
df[df.room_class=="Standard"].occ_pct.mean()

**2. Print average occupancy rate per city**

In [ ]:
df_hotels.head(3)

In [ ]:
df = pd.merge(df, df_hotels, on="property_id")
df.head(3)

In [ ]:
df.groupby("city")["occ_pct"].mean()

**3. When was the occupancy better? Weekday or Weekend?**

In [ ]:
df_date.head(3)

In [ ]:
df = pd.merge(df, df_date, left_on="check_in_date", right_on="date")
df.head(3)

In [ ]:
df.groupby("day_type")["occ_pct"].mean().round(2)

**4: In the month of June, what is the occupancy for different cities**

In [ ]:
df_june_22 = df[df["mmm yy"]=="Jun 22"]
df_june_22.head(4)

In [ ]:
df_june_22.groupby('city')['occ_pct'].mean().round(2).sort_values(ascending=False)

In [ ]:
df_june_22.groupby('city')['occ_pct'].mean().round(2).sort_values(ascending=False).plot(kind="bar")

**5: We got new data for the month of august. Append that to existing data**

In [ ]:
df_august = pd.read_csv("datasets/new_data_august.csv")
df_august.head(3)

In [ ]:
df_august.columns

In [ ]:
df.columns

In [ ]:
df_august.shape

In [ ]:
df.shape

In [ ]:
latest_df = pd.concat([df, df_august], ignore_index = True, axis = 0)
latest_df.tail(10)

In [ ]:
latest_df.shape

**6. Print revenue realized per city**

In [ ]:
df_bookings.head()

In [ ]:
df_hotels.head(3)

In [ ]:
df_bookings_all = pd.merge(df_bookings, df_hotels, on="property_id")
df_bookings_all.head(3)

In [ ]:
df_bookings_all.groupby("city")["revenue_realized"].sum()

**7. Print month by month revenue**

In [ ]:
df_date.head(3)

In [ ]:
df_date["mmm yy"].unique()

In [ ]:
df_bookings_all.head(3)

In [ ]:
df_date.info()

In [ ]:
df_date["date"] = pd.to_datetime(df_date["date"])
df_date.head(3)

In [ ]:
df_bookings_all.info()

In [ ]:
df_bookings_all["check_in_date"] = pd.to_datetime(df_bookings_all["check_in_date"])
df_bookings_all.head(4)

In [ ]:
df_bookings_all = pd.merge(df_bookings_all, df_date, left_on="check_in_date", right_on="date")
df_bookings_all.head(3)

In [ ]:
df_bookings_all.groupby("mmm yy")["revenue_realized"].sum()